
**Ma chu hang: HP, RW**

**Kho xuat: kho tot (tru online)**


In [1]:
import glob
import os

import numpy as np
import pandas as pd

# setting notebook
pd.set_option("display.max_columns", None)

In [2]:
# go to data-joined folder
%cd data-joined

C:\Users\user\Desktop\gte\notebooks\data-joined


In [3]:
%run xuat_combined.ipynb 
%run po_combined.ipynb 
%run dmsp.ipynb
%run dmkh.ipynb

#go back the tree
%cd ..

C:\Users\user\Desktop\gte\notebooks


In [4]:
# make copy
xuat = xuat_joined.copy()
po = po_joined.copy()
sp = dmsp_full.copy()
kh = dmkh_full.copy()

# A. PO

## 01. select cols

In [5]:
# select use cols
cols_used = ['id',
 'id_account',
 'stakeholder',
 'receive_date',
 'delivery_deadline',
 'delivery_date',
 'invoice_date',
 'pickup_note',
 'po_status',
 'warehouse',
 'invoice_type']

po_selected_cols = po[cols_used]

#### replace label

In [6]:
po_replace = po_selected_cols.copy()

#Replace 'Chua thuc hien' => 'Da nhat hang'
po_replace.loc[(po_replace['po_status'].str.contains('Chưa thực hiện')) 
                & (po_replace['pickup_note'].str.contains('Đã nhặt hàng')),'po_status'] = 'Đã nhặt hàng'

#Replace 'De xuat' => 'Don'
po_replace['invoice_type'] = po_replace['invoice_type'].str.replace(r'Đề xuất', 'Đơn', regex=True)

## 02. impute

In [7]:
# make a copy
po_cal = po_replace.copy()

# create month
po_cal['month_receive'] = pd.DatetimeIndex(po_cal['receive_date']).month


#### day return

In [8]:
# impute date diff invoice
diff_ct_yes = (po_cal['invoice_date'] - po_cal['delivery_date']).dt.days #da hoan
diff_ct_no = (pd.Timestamp.now().normalize() - po_cal['delivery_date']).dt.days #chua hoan

conditions = [
    po_cal['po_status']=='Đã bàn giao chứng từ',
    po_cal['po_status']=='Đã giao hàng'
]
choices = [diff_ct_yes, diff_ct_no]

po_cal['datediff_invoice'] = np.select(conditions, choices, default=np.nan)


#### day delivery

In [9]:
# impute date diff delivery
diff_deli = (po_cal['delivery_date'] - po_cal['receive_date']).dt.days
po_cal['datediff_delivery'] = np.where(diff_deli<0, np.nan, diff_deli)

## 03. filter

In [10]:
po_filter = po_cal.copy()

#Filter Warehouse not contains
values_to_remove = [
    'Online', 'Upbase',
    'Hyperion','hư hỏng','xử lý','Hư Hỏng','khuyến mãi','Tồn hàng','hàng hóa', 'Kho treo hàng'] 

pattern = '|'.join(values_to_remove)

po_filter_kho = po_filter.loc[~po_filter['warehouse'].str.contains(pattern, case=False)]

#Filter Ma chu hang HP, RW
values_to_filter = ['HP', 'RW'] 

pattern_mch = '|'.join(values_to_filter)

po_filter_mch = po_filter_kho.loc[po_filter_kho['stakeholder'].str.contains(pattern_mch, case=False)]
po_processed = po_filter_mch.copy()

# B. BCKT

## 01. merge dmsp
merge to dmsp to impute some metrics about po

In [11]:
# merge
xuat_dmsp = xuat.merge(sp, left_on='product_code', right_on='product_code', how='left')


## 02. impute

In [12]:

# impute cbm, weight
xuat_dmsp['cbm_unit'] = xuat_dmsp['cbm_unit']*xuat_dmsp['quantity']
xuat_dmsp['GrossWeightProductg'] = xuat_dmsp['GrossWeightProductg']*xuat_dmsp['quantity']

# aggregate 
xuat_dmsp = xuat_dmsp.groupby(['id', 'stakeholder']).agg(
      {
        'cbm_unit': 'sum',
        'GrossWeightProductg': 'sum',
        'quantity': 'sum',
        'product_code': 'count',
        'sub_total': 'sum'
      }
)

# rename cols
dict_rename = {
    'cbm_unit': 'cbm',
     'GrossWeightProductg': 'weight_g',
     'product_code': 'sku',
     'sub_total': 'amount'
}

xuat_dmsp.rename(columns=dict_rename, inplace=True)
xuat_processed = xuat_dmsp.copy()

# C. Create data

In [13]:
# Create data region
data = [
    ["Kho Hà Nội", "Miền Bắc"],
    ["Kho Phạm Văn Đồng", "Miền Nam"],
    ["Kho Bình Tân", "Miền Nam"],
    ["Kho Bùi Thị Xuân", "Miền Nam"],
    ["Kho Đà Nẵng", "Miền Trung"],
    ["Kho Thuận An", "Miền Nam"],
    ["Kho Bình Hòa", "Miền Nam"],
    ["Kho thuê dịch vụ - Daso", "Miền Nam"],
    ["Kho Woojin Bắc Ninh", "Miền Bắc"],
    ["Kho LogisValley - Long Biên", "Miền Bắc"],
    ["Kho thuê ngoài Emskip Q9", "Miền Nam"],
    ["Kho LogisticValey - Long Biên", "Miền Bắc"],
    ["Kho Hateco Long Biên", "Miền Bắc"],
    ["Kho Gamuda Hà Nội", "Miền Bắc"],
    ["Kho thuê dịch vụ Daso", "Miền Nam"],
    ['Kho LogisticValey Long Biên',"Miền Bắc"],
    
    # online
    ['Kho Online PVD', 'online'],
    ['Kho Online Hà Nội', 'online'],
    ['Kho Online Đà Nẵng', 'online'],
    ['Kho Online Nơ Trang Long', 'online'],
    ['Kho Upbase Nơ Trang Long', 'online'],
    ['Kho Online Upbase Hà Nội', 'online'],
    ['Kho Online Upbase PVD', 'online']
]

df_rgn = pd.DataFrame(data, columns=["warehouse", "region"])

# Create inside, outside of city
data_in_sub = [
    ["Kho Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho Phạm Văn Đồng", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bình Tân", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bùi Thị Xuân", "Bình Dương", "Nội thành"],
    ["Kho Đà Nẵng", "Đà Nẵng", "Nội thành"],
    ["Kho Thuận An", "Bình Dương", "Nội thành"],
    ["Kho Bình Hòa", "Hồ Chí Minh", "Nội thành"],
    ["Kho thuê dịch vụ - Daso", "Hồ Chí Minh", "Nội thành"],
    ["Kho Woojin Bắc Ninh", "Hà Nội", "Nội thành"],
    ["Kho LogisValley - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho thuê ngoài Emskip Q9", "Hồ Chí Minh", "Nội thành"],
    ["Kho LogisticValey - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Hateco Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Gamuda Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho thuê dịch vụ Daso", "Hồ Chí Minh", "Nội thành"],
    ['Kho LogisticValey Long Biên',"Hà Nội", "Nội thành"],
    
    # online
    ['Kho Online PVD', 'online', 'online'],
    ['Kho Online Hà Nội', 'online', 'online'],
    ['Kho Online Đà Nẵng', 'online', 'online'],
    ['Kho Online Nơ Trang Long', 'online', 'online'],
    ['Kho Upbase Nơ Trang Long', 'online', 'online'],
    ['Kho Online Upbase Hà Nội', 'online', 'online'],
    ['Kho Online Upbase PVD', 'online', 'online']
]

df_in_sub = pd.DataFrame(data_in_sub, columns=["warehouse", "destination", "in_out_city"])

# D. merge all

In [14]:
# merge bckt
po_xuat = po_processed.merge(
    xuat_processed, on=['id', 'stakeholder'], how="left")

# merge region
po_region = po_xuat.merge(df_rgn, on="warehouse", how="left")

# merge DMKH
po_kh = po_region.merge(
    kh[['id_account', 'account_name', 'destination']], on="id_account", how="left")

# merge inner/suburbs data
po_insub = po_kh.merge(df_in_sub, on=["warehouse", "destination"], how="left")


In [15]:
# 'ngoai thanh' if destionation is null
po_insub.loc[~po_insub['destination'].isna(), "in_out_city"] = po_insub['in_out_city'].fillna("Ngoại thành")

# impute return invoice class
conditions = [
    po_insub["datediff_invoice"] < 0,
    po_insub["datediff_invoice"] <= 6,
    po_insub["datediff_invoice"] <= 14,
    po_insub["datediff_invoice"] <= 30,
    po_insub["datediff_invoice"] > 30,
]
choices = ["invalid", "0 - 6 ngày", "7 - 14 ngày", "15 - 30 ngày", "Trễ"]

po_insub["returninvoice_class"] = np.select(conditions, choices, default=np.nan)
po_run = po_insub.copy()
po_run['id'] = po_run['id'] + po_run['stakeholder']


# E. export

In [16]:
import datetime
file_path = '../data/processed/'
namefile = f'PO_{datetime.datetime.now().strftime("%H%M_%Y%m%d")}.csv'
po_run.to_csv(file_path+namefile, index=False, encoding="utf-8-sig")


In [17]:
po_run[po_run['region'].isnull()]['warehouse'].unique().tolist()

[]